In [1]:
import sys
import spotipy
import json
import webbrowser
import spotipy.util as util
from json.decoder import JSONDecodeError


In [2]:
# Get user name


# Username: yts1nrqgvqjmau92w6fq1y3kx
username='yts1nrqgvqjmau92w6fq1y3kx'
scope = 'user-library-read'
client_id='78c22596fdaf488090979ee83028d942'
client_secret='58a6bf1864b34e2682d9d57487e1ec60'
redirect_uri='http://localhost/'

util.prompt_for_user_token(username,scope,client_id,client_secret,redirect_uri)
   


'BQDqhNkyjQ4KLzLyYSNELO2fmF6SB3m_369FUyovWpKA_H4RPzbBdXd4Hbrfy1HJ-8aPYU8khbGxRn749txIC6cHkBW4CRpkHo3XKlr5tp_bEBqC8DQ9ayD4OqKiZpPEPqs8e_v_TSQqdGeCu7EyyyHVlzvnm43ppgm8zWPO6k-8W-OVEjM'

## Prints the album and 

In [3]:

#TEST

import sys
import spotipy
import spotipy.util as util

scope = 'user-library-read'

username='yts1nrqgvqjmau92w6fq1y3kx'
client_id='78c22596fdaf488090979ee83028d942'
client_secret='58a6bf1864b34e2682d9d57487e1ec60'
redirect_uri='http://localhost/'

token = util.prompt_for_user_token(username,scope,client_id,client_secret,redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
    results = sp.current_user_saved_tracks()
    for item in results['items']:
        track = item['track']
        print( track['name'] + ' - ' + track['artists'][0]['name'])
else:
    print("Can't get token for", username)

Delicate - Taylor Swift
Aftergold - Big Wild


## Prints the audio features

In [4]:
import spotipy
import spotipy.oauth2

credentials = spotipy.oauth2.SpotifyClientCredentials(client_id, client_secret)
spotify = spotipy.Spotify(client_credentials_manager=credentials)
#print(spotify.audio_features(['4uLU6hMCjMI75M1A2tKUQC']))
#print(spotify.audio_features(['https://open.spotify.com/playlist/37i9dQZF1Ejh1DqQpLOGMI']))
#print(spotify.audio_features(['4JpKVNYnVcJ8tuMKjAj50A,2NRANZE9UCmPAS5XVbXL40,24JygzOLM0EmRQeGtFcIcG']))
#print(spotify.audio_analysis('4JpKVNYnVcJ8tuMKjAj50A'))

#https://open.spotify.com/album/3idhSBAaROCp9U4fQCj4Xs
print(spotify.audio_features(['party']))



[None]


### TEST

In [6]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,1000,50):
    track_results = sp.search(q='year:2018', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

    
stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 1.602753908847653


In [7]:

import pandas as pd
df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

(100, 4)


,artist_name,track_name,track_id,popularity
0,Post Malone,Wow.,6MWtB6iiXyIwun0YzU6DFP,98
1,Lady Gaga,Shallow,2VxeLyX666F8uXCJ0dZF8B,96
2,Alec Benjamin,Let Me Down Slowly,2qxmye6gAegTMjLKEBoR3d,93
3,Ava Max,Sweet but Psycho,25sgk305KZfyuqVBQIahim,96
4,Halsey,Without Me,5p7ujcrUXASCNwRaWNHR1C,96


In [8]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()



df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)



# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()


df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()

df_tracks.shape

(100, 4)

In [9]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 0.45664954204868735


In [12]:
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 100


In [13]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (100, 18)


,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.1630,https://api.spotify.com/v1/audio-analysis/6MWt...,0.833,149520,0.539,6MWtB6iiXyIwun0YzU6DFP,0.000002,11,0.1010,-7.399,0,0.1780,99.947,4,https://api.spotify.com/v1/tracks/6MWtB6iiXyIw...,audio_features,spotify:track:6MWtB6iiXyIwun0YzU6DFP,0.385
1,0.3710,https://api.spotify.com/v1/audio-analysis/2Vxe...,0.572,215733,0.385,2VxeLyX666F8uXCJ0dZF8B,0.000000,7,0.2310,-6.362,1,0.0308,95.799,4,https://api.spotify.com/v1/tracks/2VxeLyX666F8...,audio_features,spotify:track:2VxeLyX666F8uXCJ0dZF8B,0.323
2,0.7400,https://api.spotify.com/v1/audio-analysis/2qxm...,0.652,169354,0.557,2qxmye6gAegTMjLKEBoR3d,0.000000,1,0.1240,-5.714,0,0.0318,150.073,4,https://api.spotify.com/v1/tracks/2qxmye6gAegT...,audio_features,spotify:track:2qxmye6gAegTMjLKEBoR3d,0.483
3,0.0691,https://api.spotify.com/v1/audio-analysis/25sg...,0.719,187436,0.704,25sgk305KZfyuqVBQIahim,0.000000,1,0.1660,-4.724,1,0.0476,133.002,4,https://api.spotify.com/v1/tracks/25sgk305KZfy...,audio_features,spotify:track:25sgk305KZfyuqVBQIahim,0.628
4,0.2970,https://api.spotify.com/v1/audio-analysis/5p7u...,0.752,201661,0.488,5p7ujcrUXASCNwRaWNHR1C,0.000009,6,0.0936,-7.050,1,0.0705,136.041,4,https://api.spotify.com/v1/tracks/5p7ujcrUXASC...,audio_features,spotify:track:5p7ujcrUXASCNwRaWNHR1C,0.533


In [14]:
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 18 columns):
acousticness        100 non-null float64
analysis_url        100 non-null object
danceability        100 non-null float64
duration_ms         100 non-null int64
energy              100 non-null float64
id                  100 non-null object
instrumentalness    100 non-null float64
key                 100 non-null int64
liveness            100 non-null float64
loudness            100 non-null float64
mode                100 non-null int64
speechiness         100 non-null float64
tempo               100 non-null float64
time_signature      100 non-null int64
track_href          100 non-null object
type                100 non-null object
uri                 100 non-null object
valence             100 non-null float64
dtypes: float64(9), int64(4), object(5)
memory usage: 14.1+ KB


In [15]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(100, 14)

In [16]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (100, 14)


,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Post Malone,Wow.,6MWtB6iiXyIwun0YzU6DFP,98,0.1630,0.833,149520,0.539,0.000002,11,0.1010,-7.399,0,0.1780,99.947,4,0.385
1,Lady Gaga,Shallow,2VxeLyX666F8uXCJ0dZF8B,96,0.3710,0.572,215733,0.385,0.000000,7,0.2310,-6.362,1,0.0308,95.799,4,0.323
2,Alec Benjamin,Let Me Down Slowly,2qxmye6gAegTMjLKEBoR3d,93,0.7400,0.652,169354,0.557,0.000000,1,0.1240,-5.714,0,0.0318,150.073,4,0.483
3,Ava Max,Sweet but Psycho,25sgk305KZfyuqVBQIahim,96,0.0691,0.719,187436,0.704,0.000000,1,0.1660,-4.724,1,0.0476,133.002,4,0.628
4,Halsey,Without Me,5p7ujcrUXASCNwRaWNHR1C,96,0.2970,0.752,201661,0.488,0.000009,6,0.0936,-7.050,1,0.0705,136.041,4,0.533


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 17 columns):
artist_name         100 non-null object
track_name          100 non-null object
track_id            100 non-null object
popularity          100 non-null int64
acousticness        100 non-null float64
danceability        100 non-null float64
duration_ms         100 non-null int64
energy              100 non-null float64
instrumentalness    100 non-null float64
key                 100 non-null int64
liveness            100 non-null float64
loudness            100 non-null float64
mode                100 non-null int64
speechiness         100 non-null float64
tempo               100 non-null float64
time_signature      100 non-null int64
valence             100 non-null float64
dtypes: float64(9), int64(5), object(3)
memory usage: 14.1+ KB


In [18]:
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence


In [ ]:
df.to_csv('SpotifyAudioFeatures08082018.csv')